In [ ]:
library(tidyverse)
if (!dir.exists("intermediate")) dir.create("intermediate", recursive = TRUE)
if (!exists("mdpre")) mdpre <- function(x) { print(x) }
if (!exists("ggmd"))  ggmd  <- function(p) { print(p) }

In [ ]:
patient_names <- read.csv("patient_names.csv")
patient_properties <- read.csv("patient_properties.csv")
patient_names$BIRTHDATE <- as.Date(patient_names$BIRTHDATE, format = "%m/%d/%Y")
patient_names$DEATHDATE <- as.Date(patient_names$DEATHDATE, format = "%m/%d/%Y")
print(head(patient_names, 10))
print(head(patient_properties, 10))

In [ ]:
patient_properties_wide <- patient_properties %>%
  pivot_wider(
    names_from = "property",
    values_from = "value"
  )
print(head(patient_properties_wide, 10))

In [ ]:
joined_data <- left_join(patient_names, patient_properties_wide, by = "ID")
print(head(joined_data,10))

In [ ]:
joined_data <- joined_data %>%
  mutate(
    FIRST = str_replace_all(FIRST, "[^a-zA-Z]", ""),
    LAST = str_replace_all(LAST, "[^a-zA-Z]", "")
  )
print(head(joined_data, 10))

In [ ]:
mdpre <- function(x){
  cat("\n```\n")
  print(x)
  cat("\n```\n")
}
categorical_cols <- c("CITY", "STATE", "MARITAL", "RACE", "ETHNICITY", "GENDER")
for (col_name in categorical_cols) {
  
  cat("\n***\n")
  cat(paste("Counts for column:", col_name))
  cat("\n***\n")
  
  counts <- joined_data %>%
    count(!!sym(col_name), sort = TRUE)
  mdpre(counts)
}

In [ ]:
joined_data$MARITAL[joined_data$MARITAL %in% c("Fine", "male")] <- NA
library(dplyr)
joined_data <- joined_data %>%
  mutate(RACE = recode(RACE, "asiann" = "asian"))
joined_data <- joined_data %>%
  mutate(ETHNICITY = recode(ETHNICITY,
                            "nonhispani" = "nonhispanic",
                            "hispani"    = "hispanic"))

joined_data <- joined_data %>%
  mutate(GENDER = recode(GENDER,
                         "Female" = "F",
                         "female" = "F",
                         "Male"   = "M"))


In [ ]:
mdpre <- function(x){
  cat("\n```\n")
  print(x)
  cat("\n```\n")
}
categorical_cols <- c("CITY", "STATE", "MARITAL", "RACE", "ETHNICITY", "GENDER")
for (col_name in categorical_cols) {
  
  cat("\n***\n")
  cat(paste("Counts for column:", col_name))
  cat("\n***\n")
  
  counts <- joined_data %>%
    count(!!sym(col_name), sort = TRUE)
  mdpre(counts)
}
if (col_name %in% c("BIRTHDATE")) {
x <- as.Date(x, format = "%m/%d/%y")
x <- if_else(!is.na(x) & x > as.Date("2020-01-01"),
x - years(100),
x)
return(x)
}


In [ ]:

df<-joined_data
ggplot(df, aes(x = BIRTHDATE, y = MARITAL, color = MARITAL)) +
  geom_jitter(alpha = 0.6, width = 0.2, height = 0.2)


In [ ]:
if (col_name %in% c("DEATHDATE")) {
x <- as.Date(x, format = "%m/%d/%y")
x <- if_else(!is.na(x) & x > Sys.Date(),
x - years(100),
x)
return(x)
}
df <- df %>%
  mutate(
    Age = floor(
      time_length(
        interval(start = BIRTHDATE, end = ifelse(is.na(DEATHDATE), Sys.Date(), DEATHDATE)),
        unit = "years"
      )
    )
  )